In [1]:
!python -V

Python 3.9.12


In [1]:
import pandas as pd 
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("Mlflow_cooking")

2025/09/19 08:40:28 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/19 08:40:28 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/MLops-cookin-/02-MLflow/mlruns/1', creation_time=1758188410351, experiment_id='1', last_update_time=1758188410351, lifecycle_stage='active', name='Mlflow_cooking', tags={}>

In [3]:
# making it a function for easy access
def read_dataframe(filename):
    df=pd.read_parquet(filename)

    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    df['duration']=df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df=df[(df.duration >= 1.0) & (df.duration <= 62)]

    categorical=['PULocationID','DOLocationID']
    #numerical=['trip_distance']

    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train=read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
df_val=read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

In [5]:
len(df_train), len(df_val)

(74049, 62105)

In [6]:
categorical=['PULocationID','DOLocationID']
numerical=['trip_distance']

dv=DictVectorizer()

train_dicts=df_train[categorical + numerical].to_dict(orient="records")
X_train=dv.fit_transform(train_dicts)

val_dicts=df_val[categorical + numerical].to_dict(orient="records")
X_val=dv.transform(val_dicts)

In [7]:
target="duration"
y_train=df_train[target].values
y_val=df_val[target].values

In [8]:
lr=LinearRegression()
lr.fit(X_train, y_train)


y_pred=lr.predict(X_val)


mean_squared_error(y_val, y_pred, squared=False)

10.648663068172787

In [8]:
# trying the feature combination 
df_train["PU_OD"]=df_train["PULocationID"] + '_' + df_train['DOLocationID']
df_val["PU_OD"]=df_val["PULocationID"] + '_' + df_val['DOLocationID']

In [9]:
df_val

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration,PU_OD
0,2,2021-02-01 00:34:03,2021-02-01 00:51:58,N,1.0,130,205,5.0,3.66,14.00,...,10.00,0.0,None,0.3,25.30,1.0,1.0,0.00,17.916667,130_205
1,2,2021-02-01 00:04:00,2021-02-01 00:10:30,N,1.0,152,244,1.0,1.10,6.50,...,0.00,0.0,None,0.3,7.80,2.0,1.0,0.00,6.500000,152_244
2,2,2021-02-01 00:18:51,2021-02-01 00:34:06,N,1.0,152,48,1.0,4.93,16.50,...,0.00,0.0,None,0.3,20.55,2.0,1.0,2.75,15.250000,152_48
3,2,2021-02-01 00:53:27,2021-02-01 01:11:41,N,1.0,152,241,1.0,6.70,21.00,...,0.00,0.0,None,0.3,22.30,2.0,1.0,0.00,18.233333,152_241
4,2,2021-02-01 00:57:46,2021-02-01 01:06:44,N,1.0,75,42,1.0,1.89,8.50,...,2.45,0.0,None,0.3,12.25,1.0,1.0,0.00,8.966667,75_42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64567,2,2021-02-28 22:19:00,2021-02-28 22:29:00,None,NaN,129,7,NaN,2.63,10.04,...,0.00,0.0,None,0.3,10.34,NaN,NaN,NaN,10.000000,129_7
64568,2,2021-02-28 23:18:00,2021-02-28 23:27:00,None,NaN,116,166,NaN,1.87,8.33,...,1.89,0.0,None,0.3,10.52,NaN,NaN,NaN,9.000000,116_166
64569,2,2021-02-28 23:44:00,2021-02-28 23:58:00,None,NaN,74,151,NaN,2.40,12.61,...,0.00,0.0,None,0.3,12.91,NaN,NaN,NaN,14.000000,74_151
64570,2,2021-02-28 23:07:00,2021-02-28 23:14:00,None,NaN,42,42,NaN,1.11,11.95,...,0.00,0.0,None,0.3,15.00,NaN,NaN,NaN,7.000000,42_42


In [10]:
categorical=["PU_OD"]#'PULocationID','DOLocationID']
numerical=['trip_distance']

dv=DictVectorizer()

train_dicts=df_train[categorical + numerical].to_dict(orient="records")
X_train=dv.fit_transform(train_dicts)

val_dicts=df_val[categorical + numerical].to_dict(orient="records")
X_val=dv.transform(val_dicts)

In [11]:
target="duration"
y_train=df_train[target].values
y_val=df_val[target].values

In [13]:
lr=LinearRegression()
lr.fit(X_train, y_train)


y_pred=lr.predict(X_val)


mean_squared_error(y_val, y_pred, squared=False)

7.62964129101857

In [14]:
#starting with the mlflow 
with mlflow.start_run():
    
    #giving the user name and tag
    mlflow.set_tag("Beginner", "Anith")
    
    #giving the train data path and val data path
    mlflow.log_param("Train_data_path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("val_data_path", "./data/green_tripdata_2021-02.csv")
    
    alpha=0.1
    #logging the alpha params 
    mlflow.log_param("Alpha", alpha)
    
    lr=Lasso(alpha=0.001)
    lr.fit(X_train, y_train)


    y_pred=lr.predict(X_val)

    #appending the rmse score of the each run in mlflows
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    # uploads your local file in mlflow 
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [53]:
lr=Ridge(alpha=0.01)
lr.fit(X_train, y_train)


y_pred=lr.predict(X_val)


mean_squared_error(y_val, y_pred, squared=False)

11.548802761028712

In [148]:

lr=LinearRegression()
lr.fit(X_train, y_train)


y_pred=lr.predict(X_val)


mean_squared_error(y_val, y_pred, squared=False)

7.629637511286329

In [149]:
# saving the model to a folder
with open ('models/lin_reg.bin','wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
# trying this model to xgboost
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [19]:
# converting the data into a special data structure used by XGBoost
train= xgb.DMatrix(X_train, label=y_train)
valid=xgb.DMatrix(X_val, label=y_val)

In [40]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model","xgboost")
        mlflow.log_params(params)
        booster=xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )
        y_pred=booster.predict(valid)
        rmse=mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        
    return {"loss":rmse, "status": STATUS_OK}
        

In [42]:
search_space={
    "max_depth":scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate':hp.loguniform('learning_rate',-3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 13,
}

best_result=fmin(
fn=objective,
space=search_space,
algo=tpe.suggest,
max_evals=10,
trials=Trials())

  0%|                                                                                                                  | 0/10 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:51:45] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.95557                                                                                                                                  
[1]	validation-rmse:11.52265                                                                                                                                  
[2]	validation-rmse:11.12259                                                                                                                                  
[3]	validation-rmse:10.75218                                                                                                                                  
[4]	validation-rmse:10.41068                                                                                                                                  
[5]	validation-rmse:10.09595                                                                                                                                  
[6]	validation-rmse:9.80637                   

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:55:34] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.27188                                                                                                                                  
[1]	validation-rmse:8.90600                                                                                                                                   
[2]	validation-rmse:8.06649                                                                                                                                   
[3]	validation-rmse:7.56573                                                                                                                                   
[4]	validation-rmse:7.26946                                                                                                                                   
[5]	validation-rmse:7.09072                                                                                                                                   
[6]	validation-rmse:6.97620                   

...

[0]	validation-rmse:7.59315                                                                                                                                   
[1]	validation-rmse:6.92893                                                                                                                                   
[2]	validation-rmse:6.80222                                                                                                                                   
[3]	validation-rmse:6.76968                                                                                                                                   
[4]	validation-rmse:6.75497                                                                                                                                   
[5]	validation-rmse:6.75011                                                                                                                                   
[6]	validation-rmse:6.74738                   

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:58:39] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.04179                                                                                                                                  
[1]	validation-rmse:8.62782                                                                                                                                   
[2]	validation-rmse:7.82110                                                                                                                                   
[3]	validation-rmse:7.36948                                                                                                                                   
[4]	validation-rmse:7.12303                                                                                                                                   
[5]	validation-rmse:6.98389                                                                                                                                   
[6]	validation-rmse:6.89963                   

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:58:48] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.85590                                                                                                                                  
[1]	validation-rmse:11.34035                                                                                                                                  
[2]	validation-rmse:10.87186                                                                                                                                  
[3]	validation-rmse:10.44799                                                                                                                                  
[4]	validation-rmse:10.06458                                                                                                                                  
[5]	validation-rmse:9.71917                                                                                                                                   
[6]	validation-rmse:9.40634                   

In [25]:
# train the model with best params
params={
"learning_rate": 0.14988312150619953,
"max_depth":66,
"min_child_weight":1.0531411801474737,
"objective":"reg:linear",
"reg_alpha":0.01980913796072851,
"reg_lambda":0.042524169343261004,
"seed":13}

mlflow.xgboost.autolog(disable=True)

booster=xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=50)
        


/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:13:54] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.19666
[1]	validation-rmse:10.20824
[2]	validation-rmse:9.41863
[3]	validation-rmse:8.79502
[4]	validation-rmse:8.30440
[5]	validation-rmse:7.92164
[6]	validation-rmse:7.62675
[7]	validation-rmse:7.39581
[8]	validation-rmse:7.21975
[9]	validation-rmse:7.08243
[10]	validation-rmse:6.97722
[11]	validation-rmse:6.89439
[12]	validation-rmse:6.82987
[13]	validation-rmse:6.77986
[14]	validation-rmse:6.74091
[15]	validation-rmse:6.70589
[16]	validation-rmse:6.68047
[17]	validation-rmse:6.65804
[18]	validation-rmse:6.63990
[19]	validation-rmse:6.62430
[20]	validation-rmse:6.61180
[21]	validation-rmse:6.60086
[22]	validation-rmse:6.59079
[23]	validation-rmse:6.58196
[24]	validation-rmse:6.57582
[25]	validation-rmse:6.56927
[26]	validation-rmse:6.56379
[27]	validation-rmse:6.55836
[28]	validation-rmse:6.55499
[29]	validation-rmse:6.55148
[30]	validation-rmse:6.54875
[31]	validation-rmse:6.54710
[32]	validation-rmse:6.54539
[33]	validation-rmse:6.54376
[34]	validation-rmse:6

In [31]:
# logging the model using the mlflow 

#starting the mlflow
with mlflow.start_run():
    
    train= xgb.DMatrix(X_train, label=y_train)
    valid=xgb.DMatrix(X_val, label=y_val)
    
    
    best_params={
        "learning_rate": 0.14988312150619953,
        "max_depth":66,
        "min_child_weight":1.0531411801474737,
        "objective":"reg:linear",
        "reg_alpha":0.01980913796072851,
        "reg_lambda":0.042524169343261004,
        "seed":13}
    #saving the best params in mlflow
    mlflow.log_params(best_params)
    
    booster=xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=50)
    
    y_pred = booster.predict(valid)
    rmse=mean_squared_error(y_val, y_pred, squared=False)
    #saving the metrics score in mlflow
    mlflow.log_metric("rmse",rmse)
    
    with open("models/preprocessor.b", 'wb') as f_out:
        pickle.dump(dv, f_out)
    #saving the preprocessing data in mlflow
    mlflow.log_artifact("models/preprocessor.b", artifact_path='preprocessor')
    
    #saving the model in mlflow
    mlflow.xgboost.log_model(booster, artifact_path="model_mlflow")

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:24:46] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.19666
[1]	validation-rmse:10.20824
[2]	validation-rmse:9.41863
[3]	validation-rmse:8.79502
[4]	validation-rmse:8.30440
[5]	validation-rmse:7.92164
[6]	validation-rmse:7.62675
[7]	validation-rmse:7.39581
[8]	validation-rmse:7.21975
[9]	validation-rmse:7.08243
[10]	validation-rmse:6.97722
[11]	validation-rmse:6.89439
[12]	validation-rmse:6.82987
[13]	validation-rmse:6.77986
[14]	validation-rmse:6.74091
[15]	validation-rmse:6.70589
[16]	validation-rmse:6.68047
[17]	validation-rmse:6.65804
[18]	validation-rmse:6.63990
[19]	validation-rmse:6.62430
[20]	validation-rmse:6.61180
[21]	validation-rmse:6.60086
[22]	validation-rmse:6.59079
[23]	validation-rmse:6.58196
[24]	validation-rmse:6.57582
[25]	validation-rmse:6.56927
[26]	validation-rmse:6.56379
[27]	validation-rmse:6.55836
[28]	validation-rmse:6.55499
[29]	validation-rmse:6.55148
[30]	validation-rmse:6.54875
[31]	validation-rmse:6.54710
[32]	validation-rmse:6.54539
[33]	validation-rmse:6.54376
[34]	validation-rmse:6

2025/09/19 09:25:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:25:59] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/09/19 09:26:02 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpprrrahmy/model, flavor: xgboost). Fall back to return ['xgboost==2.1.4']. Set logging level to DEBUG to see the full traceback. 
2025/09/19 09:26:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [41]:
with mlflow.start_run():
    
    train= xgb.DMatrix(X_train, label=y_train)
    valid=xgb.DMatrix(X_val, label=y_val)
    
    
    best_params={
        "learning_rate": 0.14988312150619953,
        "max_depth":66,
        "min_child_weight":1.0531411801474737,
        "objective":"reg:squarederror",
        "reg_alpha":0.01980913796072851,
        "reg_lambda":0.042524169343261004,
        "seed":13}
    #saving the best params in mlflow
    mlflow.log_params(best_params)
    
    booster=xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, "validation")],
            early_stopping_rounds=50)
    
    y_pred = booster.predict(valid)
    rmse=mean_squared_error(y_val, y_pred, squared=False)
    #saving the metrics score in mlflow
    mlflow.log_metric("rmse",rmse)
    
    with open("models/preprocessor.b", 'wb') as f_out:
        pickle.dump(dv, f_out)
    #saving the preprocessing data in mlflow
    mlflow.log_artifact("models/preprocessor.b", artifact_path='preprocessor')
    
    #saving the model in mlflow
    #mlflow.xgboost.log_model(booster, name="model_mlflow")

    mlflow.xgboost.log_model(
    booster,
    name="model_mlflow",
    pip_requirements=["xgboost==2.1.4", "scikit-learn==1.5.2"]  # example
    )

[0]	validation-rmse:11.19666
[1]	validation-rmse:10.20824
[2]	validation-rmse:9.41863
[3]	validation-rmse:8.79502
[4]	validation-rmse:8.30440
[5]	validation-rmse:7.92164
[6]	validation-rmse:7.62675
[7]	validation-rmse:7.39581
[8]	validation-rmse:7.21975
[9]	validation-rmse:7.08243


/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:49:27] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/09/19 09:49:27 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - scikit-learn (current: 1.0.2, required: scikit-learn==1.5.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/09/19 09:49:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [42]:
logged_model="runs:/d70fa3006e194ddf83c60d67c2e56182/models_mlflow"
loaded_model=mlflow.pyfunc.load_model(logged_model)

MlflowException: Failed to download artifacts from path 'models_mlflow', please ensure that the path is correct.